In [1]:
import os

import dotenv
import polars as pl
from pymongo import MongoClient, UpdateOne

dotenv.load_dotenv()

SERVER_MONGO_URI = os.getenv("MONGO_URI_SERVER")

client = MongoClient(SERVER_MONGO_URI)

In [8]:
users = tuple(user["username"] for user in client.LibreChat.users.find({}, {"username": 1, "_id": 0}))
users_df = pl.read_csv("/mnt/c/Users/OU000150/Downloads/MMC_USERS_AD.csv")
users_df = users_df.filter(pl.col("LOGIN").is_in(users), pl.col("STATUSFUNC") == "A")

In [10]:
users_to_update = {}

for user in users_df.iter_rows(named=True):
    users_to_update[user["LOGIN"]] = {
        "costCenterCode": user["CODCCUSTO"],
        "costCenterName": user["NOMECCUSTO"],
        "workPositionCode": user["CODCARGO"],
        "workPositionName": user["NOMECARGO"],
    }

operations = [
    UpdateOne(
        {"username": user},
        {"$set": users_to_update[user]},
    )
    for user in users_to_update
]

client.LibreChat.users.bulk_write(operations)

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 90, 'nModified': 30, 'nRemoved': 0, 'upserted': []}, acknowledged=True)